In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F
import os
import pandas as pd

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()


22/03/06 07:49:03 WARN Utils: Your hostname, UTHMAN-PC resolves to a loopback address: 127.0.1.1; using 172.31.68.63 instead (on interface eth0)
22/03/06 07:49:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/06 07:49:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark.version

'3.0.3'

In [4]:
os.listdir()

['data',
 'week_5_homework.ipynb',
 '03_test.ipynb',
 'download_data.sh',
 '04_pyspark.ipynb',
 'test.sh',
 'taxi+_zone_lookup.csv',
 '06_spark_sql.ipynb',
 '.ipynb_checkpoints',
 '08_rdds.ipynb',
 'head.csv',
 '05_taxi_schema.ipynb',
 '07_groupby_join.ipynb',
 'spark-warehouse']

In [11]:
!wc -l data/raw/fhvhv/2021/02/fhvhv_tripdata_2021_02.csv.gz

492772 data/raw/fhvhv/2021/02/fhvhv_tripdata_2021_02.csv.gz


In [12]:
# !head -n 1000 fhvhv_tripdata_2021-01.csv > head.csv
# !wc -l head.csv

In [13]:
df_pandas = pd.read_csv("data/raw/fhvhv/2021/02/fhvhv_tripdata_2021_02.csv.gz", nrows=10)

In [14]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [15]:
spark.createDataFrame(df_pandas).schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

In [16]:
fhvhv_schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType() , True),
    types.StructField('pickup_datetime', types.TimestampType() , True),
    types.StructField('dropoff_datetime', types.TimestampType() , True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.DoubleType(), True)
 ])

In [17]:
# x='1234'
# for i in range(len(x)):
#     print(i)

### Testing the data directories to make sure they are correct

In [18]:
year = 2021
taxi_type = 'fhvhv'

for month in range(1, 4):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/{taxi_type}/{year}/{month:02d}/'
    output_path = f'data/pq/{taxi_type}/{year}/{month:02d}/'
    print(input_path)
    print(output_path)

processing data for 2021/1
data/raw/fhvhv/2021/01/
data/pq/fhvhv/2021/01/
processing data for 2021/2
data/raw/fhvhv/2021/02/
data/pq/fhvhv/2021/02/
processing data for 2021/3
data/raw/fhvhv/2021/03/
data/pq/fhvhv/2021/03/


In [ ]:
# input_path = f'data/raw/fhvhv/fhvhv_tripdata_2021-01.csv.gz'
# output_path = f'data/pq/fhvhv/2021/01/'

year = 2021
taxi_type = 'fhvhv'

# processing data for Month 1 to 3 (Range does not include the last number in the loop for some reasons i dont understand yet)
for month in range(1, 4):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/{taxi_type}/{year}/{month:02d}/'
    output_path = f'data/pq/{taxi_type}/{year}/{month:02d}/'
    
    #Reading Compressed csv files into Spark RDD
    df_fhvhv = spark.read \
    .option("header", "true") \
    .schema(fhvhv_schema) \
    .csv(input_path)
    
    #Writing to parquet
    df_fhvhv \
    .repartition(24) \
    .write.parquet(output_path,mode='overwrite')

In [19]:
#fhvhv
df_fhvhv = spark.read.parquet('data/pq/fhvhv/2021/*/*')

In [20]:
#taxi_zone_lookup
df_zones = spark.read \
    .option("header", "true") \
    .csv('data/raw/zones/*')

In [24]:
df_zones.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [25]:
df_fhvhv.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: double (nullable = true)



In [27]:
df_fhvhv.show(5)

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02872|2021-03-25 12:13:34|2021-03-25 12:22:28|         181|         189|   null|
|           HV0003|              B02866|2021-03-31 08:36:43|2021-03-31 09:01:44|         164|         138|   null|
|           HV0005|              B02510|2021-03-20 14:07:13|2021-03-20 14:31:28|          37|         188|   null|
|           HV0005|              B02510|2021-03-06 18:05:13|2021-03-06 18:16:21|          79|         164|   null|
|           HV0003|              B02888|2021-03-28 01:27:35|2021-03-28 01:38:26|         107|         226|   null|
+-----------------+--------------------+-------------------+-------------------+

#### Adding new columns to the RDD

In [29]:
df_fhvhv \
    .withColumn('pickup_date', F.to_date(df_fhvhv.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df_fhvhv.dropoff_datetime)) \
    .withColumn('service_type', F.lit('fhvhv')) \
    .select('service_type', 'hvfhs_license_num', 'dispatching_base_num', 'pickup_date', 'pickup_datetime', 'dropoff_date', \
           'dropoff_datetime', 'PULocationID', 'DOLocationID', 'SR_Flag') \
    .show(5)

+------------+-----------------+--------------------+-----------+-------------------+------------+-------------------+------------+------------+-------+
|service_type|hvfhs_license_num|dispatching_base_num|pickup_date|    pickup_datetime|dropoff_date|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+------------+-----------------+--------------------+-----------+-------------------+------------+-------------------+------------+------------+-------+
|       fhvhv|           HV0003|              B02872| 2021-03-25|2021-03-25 12:13:34|  2021-03-25|2021-03-25 12:22:28|         181|         189|   null|
|       fhvhv|           HV0003|              B02866| 2021-03-31|2021-03-31 08:36:43|  2021-03-31|2021-03-31 09:01:44|         164|         138|   null|
|       fhvhv|           HV0005|              B02510| 2021-03-20|2021-03-20 14:07:13|  2021-03-20|2021-03-20 14:31:28|          37|         188|   null|
|       fhvhv|           HV0005|              B02510| 2021-03-06|2021-03-06 18:05:

#### Creatting a new RDD after Adding new columns(pickup_date, dropoff_date, service_type etc) to the initial RDD

In [30]:
df_fhvhv_sel = df_fhvhv \
    .withColumn('pickup_date', F.to_date(df_fhvhv.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df_fhvhv.pickup_datetime)) \
    .withColumn('service_type', F.lit('fhvhv')) \
    .select('service_type', 'hvfhs_license_num', 'dispatching_base_num', 'pickup_date', 'pickup_datetime', 'dropoff_date', \
           'dropoff_datetime', 'PULocationID', 'DOLocationID', 'SR_Flag')

In [31]:
df_fhvhv_sel.show(5)

+------------+-----------------+--------------------+-----------+-------------------+------------+-------------------+------------+------------+-------+
|service_type|hvfhs_license_num|dispatching_base_num|pickup_date|    pickup_datetime|dropoff_date|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+------------+-----------------+--------------------+-----------+-------------------+------------+-------------------+------------+------------+-------+
|       fhvhv|           HV0003|              B02872| 2021-03-25|2021-03-25 12:13:34|  2021-03-25|2021-03-25 12:22:28|         181|         189|   null|
|       fhvhv|           HV0003|              B02866| 2021-03-31|2021-03-31 08:36:43|  2021-03-31|2021-03-31 09:01:44|         164|         138|   null|
|       fhvhv|           HV0005|              B02510| 2021-03-20|2021-03-20 14:07:13|  2021-03-20|2021-03-20 14:31:28|          37|         188|   null|
|       fhvhv|           HV0005|              B02510| 2021-03-06|2021-03-06 18:05:

In [32]:
df_fhvhv_sel.groupBy('dispatching_base_num').count().show(5)
print('')
print(f'Count count of total records from 01_jan to 03_mar is {df_fhvhv_sel.count()}')

#     .orderBy('dispatching_base_num', 'count') \
#     .show()

+--------------------+-------+
|dispatching_base_num|  count|
+--------------------+-------+
|              B02876| 711785|
|              B03136|   5242|
|              B02877| 647412|
|              B02869|1415850|
|              B02883| 827348|
+--------------------+-------+
only showing top 5 rows


Count count of total records from 01_jan to 03_mar is 37749803


#### Registering Tables so they can be visible as SQL Tables

In [33]:
#fhvhv
df_fhvhv_sel.registerTempTable('fhvhv_trips_data')
#zones
df_zones.registerTempTable('zones')

##### test SQL Statement

In [34]:
spark.sql("""
SELECT 
    service_type,
    dispatching_base_num,
    count(1) as count
FROM 
    fhvhv_trips_data
GROUP BY 
    service_type, dispatching_base_num
ORDER BY
    count desc
LIMIT 10
""").show()

+------------+--------------------+--------+
|service_type|dispatching_base_num|   count|
+------------+--------------------+--------+
|       fhvhv|              B02510|10266577|
|       fhvhv|              B02764| 3174154|
|       fhvhv|              B02872| 2913418|
|       fhvhv|              B02875| 2244522|
|       fhvhv|              B02765| 1828172|
|       fhvhv|              B02869| 1415850|
|       fhvhv|              B02887| 1054524|
|       fhvhv|              B02866| 1022875|
|       fhvhv|              B02871| 1010555|
|       fhvhv|              B02864| 1008413|
+------------+--------------------+--------+



### Question 3 
#### How many taxi trips were there on February 15?, Consider only trips that started on February 15.

In [35]:
spark.sql("""
SELECT 
    service_type,
    pickup_date,
    count(1) as count
FROM 
    fhvhv_trips_data
WHERE 
    pickup_date = '2021-02-15'
GROUP BY 
    service_type, pickup_date
""").show()

+------------+-----------+------+
|service_type|pickup_date| count|
+------------+-----------+------+
|       fhvhv| 2021-02-15|367170|
+------------+-----------+------+



### Question 4
##### Now calculate the duration for each trip.
##### Trip starting on which day was the longest? 

In [38]:
spark.sql("""
SELECT 
     service_type
    ,hvfhs_license_num
    ,dispatching_base_num
    ,pickup_date
   -- ,pickup_datetime
    ,dropoff_date
    --,dropoff_datetime
    --,PULocationID
    --,DOLocationID
    --,SR_Flag
    ,round(((bigint(dropoff_datetime))-(bigint(pickup_datetime)))/(60),2) trip_duration_min_sec
FROM 
    fhvhv_trips_data
WHERE 
    pickup_date between '2021-02-01' AND '2021-02-28'
ORDER BY 
    trip_duration_min_sec DESC
LIMIT 
    1
""").show()

+------------+-----------------+--------------------+-----------+------------+---------------------+
|service_type|hvfhs_license_num|dispatching_base_num|pickup_date|dropoff_date|trip_duration_min_sec|
+------------+-----------------+--------------------+-----------+------------+---------------------+
|       fhvhv|           HV0005|              B02510| 2021-02-11|  2021-02-11|               1259.0|
+------------+-----------------+--------------------+-----------+------------+---------------------+



### Question 5
##### Now find the most frequently occurring `dispatching_base_num` in this dataset.

##### How many stages this spark job has?

- Note: the answer may depend on how you write the query,
- so there are multiple correct answers. 
- Select the one you have. 

In [40]:
spark.sql("""
SELECT 
     service_type
    ,dispatching_base_num
    ,count(1) count
FROM 
    fhvhv_trips_data
WHERE 
    pickup_date between '2021-02-01' AND '2021-02-28'
GROUP BY 
    service_type, dispatching_base_num
ORDER BY 
    count DESC
LIMIT 
    1
""").show()

+------------+--------------------+-------+
|service_type|dispatching_base_num|  count|
+------------+--------------------+-------+
|       fhvhv|              B02510|3233664|
+------------+--------------------+-------+



### Question 6
##### Find the most common pickup-dropoff pair. 

##### For example:
- "Jamaica Bay / Clinton East"
- Enter two zone names separated by a slash
- If any of the zone names are unknown (missing), use "Unknown". For example, "Unknown / Clinton East". 

In [71]:
# wk_5_q6_ans = 
spark.sql("""
--select * from (
SELECT 
     service_type
    ,NVL(b.Zone,'Unknown') ||' / ' || NVL(c.Zone,'Unknown') as pickup_dropoff_pair
    ,count(1)count
FROM 
    fhvhv_trips_data a 
    INNER JOIN   zones b
    ON a.PULocationID = b.LocationID
    INNER JOIN   zones c
    ON a.DOLocationID = c.LocationID
WHERE 
    pickup_date between '2021-02-01' AND '2021-02-28'
   -- and (b.LocationID is null or c.LocationID is null)
GROUP BY 
    service_type
    ,NVL(b.Zone,'Unknown') ||' / ' || NVL(c.Zone,'Unknown')
ORDER BY 
    count DESC
LIMIT
    100
--)
-- LOWER(pickup_dropoff_pair) like '%unknown%'
""").show()

+------------+--------------------+-----+
|service_type| pickup_dropoff_pair|count|
+------------+--------------------+-----+
|       fhvhv|East New York / E...|45041|
|       fhvhv|Borough Park / Bo...|37329|
|       fhvhv| Canarsie / Canarsie|28026|
|       fhvhv|Crown Heights Nor...|25976|
|       fhvhv|Bay Ridge / Bay R...|17934|
|       fhvhv|Jackson Heights /...|14688|
|       fhvhv|   Astoria / Astoria|14688|
|       fhvhv|Central Harlem No...|14481|
|       fhvhv|Bushwick South / ...|14424|
|       fhvhv|Flatbush/Ditmas P...|13976|
|       fhvhv|South Ozone Park ...|13716|
|       fhvhv|Brownsville / Bro...|12829|
|       fhvhv|    JFK Airport / NA|12542|
|       fhvhv|Prospect-Lefferts...|11814|
|       fhvhv|Forest Hills / Fo...|11548|
|       fhvhv|Bushwick North / ...|11491|
|       fhvhv|Bushwick South / ...|11487|
|       fhvhv|Crown Heights Nor...|11462|
|       fhvhv|Crown Heights Nor...|11342|
|       fhvhv|Prospect-Lefferts...|11308|
+------------+--------------------

In [68]:
wk_5_q6_ans.write.csv('data/raw/fhvhv/homework/wk_5_q6_ans', header=True, mode= 'overwrite')

In [ ]:
spark.sql("""
SELECT 
   *
FROM 
    zones
""").show()